## Eye disease Classification Using Grey Level Co-occurrence Matrix

## Step for classification
### 1. Explore Data Image
### 2. Preprocessing Data
### 3. Extraction Feature Using Grey level co-occurrence matrix (GLCM)
### 4. Normalize Data
### 5. Classification using Machine Learning Alghoritm

## These Classification Goals is to create model that can classify eye disease

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Explore Image Data

In these Exploring Image Data i want to make image data ready for feature extraction using GLCM, So i try some image preprocessing to make image is ready for feature extraction

in this explore image data i want to make some changes to image like
1. Crop image, so unusefull black image can dissapear from image
2. Resize image, because image data have a big dimension and i large data make a long time when processing so i resize to small image

These are some packages to show image

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops

Create method to show image, these created to easily show image using matplotlib

In [ ]:
def show_image(img, cmap='gray'):
    fig = plt.figure(figsize=(20,20))
    axes = fig.add_subplot(111)
    axes.imshow(img, cmap=cmap)

Read image using cv and convert image color from BGR(Blue, Green, Red) to RGB(Red, Green, Blue) because cv automatically make image color to BGR, and matplotlib make wierd image color when showing BGR image. So we convert BGR to RGB

In [ ]:
test_img = cv.imread('../input/cataractdataset/dataset/1_normal/NL_001.png')
test_img = cv.cvtColor(test_img, cv.COLOR_BGR2RGB)
show_image(test_img)

Convert image from RGB to Grayscale, in these stage we convert 3 dimension from RGB to 1 dimensian in Grayscale

In [ ]:
width, height, dimension = test_img.shape
print(f'Width RGB = {width}')
print(f'Height RGB = {height}')
print(f'Dimension RGB = {dimension}')

In [ ]:
test_img_gray = cv.cvtColor(test_img, cv.COLOR_RGB2GRAY)
show_image(test_img_gray)

Grayscale image doseant have dimensiaon becasue grayscale image has 1 color that have range from 0 - 255

In [ ]:
width, height = test_img_gray.shape
print(f'Width Grayscale = {width}')
print(f'Height Grayscale = {height}')
print(f'Image Shape Grayscale {test_img_gray.shape}')

### Thresholding Image (Binary Image) from grayscale that have a color range from 0-255 to Threshold Image that have color 0 or 1

In [ ]:
test_img_thresh = cv.adaptiveThreshold(test_img_gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,11,3)
show_image(test_img_thresh)

Cropping Image so unusefull black color dissapeaer from image and i can get the Region of Interest (ROI) of this Image

In [ ]:
cnts = cv.findContours(test_img_thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv.contourArea, reverse=True)
for c in cnts:
    x,y,w,h = cv.boundingRect(c)
    test_img_ROI = test_img[y:y+h, x:x+w]
    break
show_image(test_img_ROI)

Because i crop image from original RGB color so these ROI image have 3 dimensions

In [ ]:
width, height, dimension = test_img_ROI.shape
print(f'Width = {width}')
print(f'Height = {height}')
print(f'Dimension = {dimension}')

Resizing image becase 1600x1600 is to large when processing, i resize to 400x400 by divided by 4

In [ ]:
test_img_ROI_resize = cv.resize(test_img_ROI, (int(width/4), int(height/4)))
show_image(test_img_ROI_resize)

Check the reized image that have been resized

In [ ]:
width, height, dimension = test_img_ROI_resize.shape
print(f'Width = {width}')
print(f'Height = {height}')
print(f'Dimension = {dimension}')

Convert resizing image to grayscale because when extraction feature with GLCM image should have 1 dimensions like grayscale

In [ ]:
test_img_ROI_resize_gray = cv.cvtColor(test_img_ROI_resize, cv.COLOR_RGB2GRAY)
show_image(test_img_ROI_resize_gray)

Test image to feture extraction with GLCM

In [ ]:
def glcm_feature(matrix_coocurrence, featureName):
    feature = greycoprops(matrix_coocurrence, featureName)
    result = np.average(feature)
    return result
distance = 10
teta = 90

contrast_test = []
homogeneity_test = []
energy_test = []
correlation_test = []


glcm = greycomatrix(test_img_ROI_resize_gray, [distance], [teta], levels=256, symmetric=True, normed=True)
contrast_test.append(glcm_feature(glcm, 'contrast'))
homogeneity_test.append(glcm_feature(glcm, 'homogeneity'))
energy_test.append(glcm_feature(glcm, 'energy'))
correlation_test.append(glcm_feature(glcm, 'correlation'))

print(f'Homogenity : {homogeneity_test[0]}')
print(f'Correlation : {correlation_test[0]}')
print(f'Energy : {energy_test[0]}')
print(f'Contrast : {contrast_test[0]}')

# 2. Preprocessing Image

After i explore some image to be image test, and have some preprocessing image like cropping and resize. i make method to gather all preprocessing step to make image ready to feature extraction

In [ ]:
def preprocessingImage(image):
    test_img = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    test_img_gray = cv.cvtColor(test_img, cv.COLOR_RGB2GRAY)
    test_img_thresh = cv.adaptiveThreshold(test_img_gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,11,3)
    cnts = cv.findContours(test_img_thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv.contourArea, reverse=True)
    for c in cnts:
        x,y,w,h = cv.boundingRect(c)
        test_img_ROI = test_img[y:y+h, x:x+w]
        break
    test_img_ROI_resize = cv.resize(test_img_ROI, (width, height))
    test_img_ROI_resize_gray = cv.cvtColor(test_img_ROI_resize, cv.COLOR_RGB2GRAY)
    
    return test_img_ROI_resize_gray

# 3. Feature Extraction

Create some variable that used for feature extraction

In [ ]:
file_normal = 301
file_cataract = 101
file_glaucoma = 102
file_retina = 101
width, height = 400, 400
distance = 10
teta = 90
data_eye = np.zeros((5, 601))
count = 0
indextable = ['contrast', 'homogenity', 'energy', 'correlation', 'Label']

normal_dataset_path = '../input/cataractdataset/dataset/1_normal/'
cataract_dataset_path = '../input/cataractdataset/dataset/2_cataract/'
glaucoma_dataset_path = '../input/cataractdataset/dataset/2_glaucoma/'
retina_dataset_path = '../input/cataractdataset/dataset/3_retina_disease/'

in these feature extraction i split these processiong based on label and i used 10 distance and 90 teta for GLCM feature extraction

In [ ]:
for file in range(1, file_normal):
    contrast = []
    homogeneity = []
    energy = []
    correlation = []
    label = 0
    image = cv.imread(f'{normal_dataset_path}/NL_{str(file).zfill(3)}.png')
    img = preprocessingImage(image)
    
    glcm = greycomatrix(img, [distance], [teta], levels=256, symmetric=True, normed=True)
    contrast.append(glcm_feature(glcm, 'contrast'))
    homogeneity.append(glcm_feature(glcm, 'homogeneity'))
    energy.append(glcm_feature(glcm, 'energy'))
    correlation.append(glcm_feature(glcm, 'correlation'))
    
    data_eye[0, count] = contrast[0]
    data_eye[1, count] = homogeneity[0]
    data_eye[2, count] = energy[0]
    data_eye[3, count] = correlation[0]
    data_eye[4, count] = label
    
    count += 1

Testing if the feature extraction is succesful

In [ ]:
data_eye

In [ ]:
for file in range(1, file_cataract):
    contrast = []
    homogeneity = []
    energy = []
    correlation = []
    label = 1
    image = cv.imread(f'{cataract_dataset_path}/cataract_{str(file).zfill(3)}.png')
    img = preprocessingImage(image)
    
    glcm = greycomatrix(img, [distance], [teta], levels=256, symmetric=True, normed=True)
    contrast.append(glcm_feature(glcm, 'contrast'))
    homogeneity.append(glcm_feature(glcm, 'homogeneity'))
    energy.append(glcm_feature(glcm, 'energy'))
    correlation.append(glcm_feature(glcm, 'correlation'))
    
    data_eye[0, count] = contrast[0]
    data_eye[1, count] = homogeneity[0]
    data_eye[2, count] = energy[0]
    data_eye[3, count] = correlation[0]
    data_eye[4, count] = label
    
    count += 1

In [ ]:
for file in range(1, file_glaucoma):
    contrast = []
    homogeneity = []
    energy = []
    correlation = []
    label = 2
    image = cv.imread(f'{glaucoma_dataset_path}/Glaucoma_{str(file).zfill(3)}.png')
    img = preprocessingImage(image)
    
    glcm = greycomatrix(img, [distance], [teta], levels=256, symmetric=True, normed=True)
    contrast.append(glcm_feature(glcm, 'contrast'))
    homogeneity.append(glcm_feature(glcm, 'homogeneity'))
    energy.append(glcm_feature(glcm, 'energy'))
    correlation.append(glcm_feature(glcm, 'correlation'))
    
    data_eye[0, count] = contrast[0]
    data_eye[1, count] = homogeneity[0]
    data_eye[2, count] = energy[0]
    data_eye[3, count] = correlation[0]
    data_eye[4, count] = label
    
    count += 1

In [ ]:
for file in range(1, file_retina):
    contrast = []
    homogeneity = []
    energy = []
    correlation = []
    label = 3
    image = cv.imread(f'{retina_dataset_path}/Retina_{str(file).zfill(3)}.png')
    img = preprocessingImage(image)
    
    glcm = greycomatrix(img, [distance], [teta], levels=256, symmetric=True, normed=True)
    contrast.append(glcm_feature(glcm, 'contrast'))
    homogeneity.append(glcm_feature(glcm, 'homogeneity'))
    energy.append(glcm_feature(glcm, 'energy'))
    correlation.append(glcm_feature(glcm, 'correlation'))
    
    data_eye[0, count] = contrast[0]
    data_eye[1, count] = homogeneity[0]
    data_eye[2, count] = energy[0]
    data_eye[3, count] = correlation[0]
    data_eye[4, count] = label
    
    count += 1

Export value of image feature extraction into csv file to make easier to see

In [ ]:
df= pd.DataFrame(np.transpose(data_eye), columns = indextable)

# 4. Normalize Data

In [ ]:
df

Data have different value so i normalize using min max scaler

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
features = df.drop(['Label'], axis='columns')
features_scaler = MinMaxScaler()
features = features_scaler.fit_transform(features)
features

Concatenate the normalization data and the label data into new dataframe

In [ ]:
data_normalization = df.copy()
data_normalization[['contrast', 'homogenity', 'energy', 'correlation']] = features
data_normalization

In [ ]:
data_normalization.describe()

In [ ]:
x = data_normalization.drop(['Label'], axis='columns')
y = data_normalization.Label

# 5. Classification using Machine Learning Alghoritm

In these stage i try to using some machine learning alghoritm with some paramater to create a better model

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20,30],
            'kernel': ['rbf','linear','poly']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10,50,100]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10,50,100]
        }
    },
    'KNN' : {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3,7,11,13]
        }
    }
    
}

In [ ]:
from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_score

SVM have the best score with 58%

In [ ]:
df_score['best_score'].max()

Split data to create confusion matrix of these svm Model

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [ ]:
model = svm.SVC(gamma='auto', C=20, kernel='poly')
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sb
y_predicted = model.predict(x_test)
cm = confusion_matrix(y_test,y_predicted)
plt.figure(figsize = (10,7))
sb.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

SVM model can only predict normal and cataract dataset

Try using neural network to create a model

In [ ]:
import tensorflow as tf
from tensorflow import keras
model_nn = keras.Sequential([
    keras.layers.Flatten(input_shape=(4,)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(4, activation='sigmoid')
])

model_nn.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model_nn.fit(x_train, y_train, epochs=50)

In [ ]:
model_nn.evaluate(x_test, y_test)
model_nn.save("eye_model.h5")
from keras.models import load_model
model1 = load_model("./eye_model.h5")
print(model1.evaluate(x_test, y_test, verbose=0))
print(model1)


In [ ]:
y_predicted = model_nn.predict(x_test)
print(y_predicted[0])
y_predicted_labels = [np.argmax(i) for i in y_predicted]
cm = tf.math.confusion_matrix(labels=y_test, predictions=y_predicted_labels)
plt.figure(figsize = (10,7))
sb.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

Neural Network Model have same similiar result like SVM model